In [1]:
#  데이터 처리 모듈
import pandas as pd
from sklearn.impute import KNNImputer
# 행렬 등 통계 연산 모듈
import numpy as np
from scipy.stats import iqr
from sklearn.impute import KNNImputer
# 지수형 표기법 e를 연속형 변환
pd.options.display.float_format = '{:.4f}'.format
# 타입 어노테이션(Any, Sequence 등의 메서드 활용)
from typing import *
import os

# 구글 드라이브 마운트
from google.colab import drive
drive.mount("/content/drive")
import os

# 조합 찾는 반복문 메서드
from itertools import permutations
from itertools import combinations

# 데이터 시각화
import matplotlib.pyplot as plt
# from matplotlib.pyplot import
import matplotlib.font_manager as fm
import missingno as msno
import seaborn as sns

Mounted at /content/drive


In [2]:
# apt-get : 패키지 설치 명령어
# -qq : 에러외의 메세지 숨기는 옵션키
!apt-get -qq install fonts-nanum

fe = fm.FontEntry(
    fname=r"/usr/share/fonts/truetype/nanum/NanumGothic.ttf",
    # ttf 파일이 저장되어 있는 경로
    name='NanumGothic')
# 이 폰트의 원하는 이름 설정
fm.fontManager.ttflist.insert(0, fe)
# Matplotlib에 폰트 추가
plt.rcParams.update({'font.size': 18, 'font.family': 'NanumGothic'})
# 폰트 설정

Selecting previously unselected package fonts-nanum.
(Reading database ... 120895 files and directories currently installed.)
Preparing to unpack .../fonts-nanum_20200506-1_all.deb ...
Unpacking fonts-nanum (20200506-1) ...
Setting up fonts-nanum (20200506-1) ...
Processing triggers for fontconfig (2.13.1-4.2ubuntu5) ...


In [3]:
class LoadGoogleDriveData():
  def __init__(self, data = None):
    self.data = data

  def loadData(self, file_path: str, file_name_extension,
               columnTF: bool, unicode: str) -> pd.DataFrame():
    self.data = pd.read_csv(os.path.join(file_path + file_name_extension),
                            index_col = columnTF,
                            na_values = "NaN",
                            encoding = unicode)
    return self.data

  # 용량이 큰 csv 파일 읽어오기(fopen - fread와 유사한 방식)
  def loadDataWithChunking(self, file_path: str, file_name_extension,
                           chunking_row_num: int, columnTF: bool, unicode: str) -> pd.DataFrame():
    chunkdata = pd.read_csv(os.path.join(file_path + file_name_extension),
                            chunksize = chunking_row_num,
                            index_col = columnTF,
                            na_values = "NaN",
                            encoding = unicode)
    self.data = list(chunkdata)
    self.data = pd.concat(self.data)

    return self.data

In [4]:
mountInstance = LoadGoogleDriveData()

### 1.오아시스 비즈니스(필지고유번호 & 법정동읍면동코드 & 외식업종)  데이터

In [5]:
CONTEST_2023 = mountInstance.loadDataWithChunking(
    file_path = "/content/drive/MyDrive/2023BigContest/data/contest data/", file_name_extension = "소상공인 매출등급 예상 대상 필지.csv",
    unicode = "cp949", chunking_row_num = 10**5, columnTF = False)

In [6]:
CONTEST_2022 = mountInstance.loadDataWithChunking(
    file_path = "/content/drive/MyDrive/2023BigContest/data/contest data/", file_name_extension = "필지단위 소상공인 매출등급 정보.csv",
    unicode = "cp949", chunking_row_num = 10**5, columnTF = False)

### 2. KCD(상권코드 & 외식업종 중분류) 소상공인 신용데이터

In [7]:
KCD_2022_2023 = mountInstance.loadData(
    file_path = "/content/drive/MyDrive/2023BigContest/data/DESILO_PLATFORM_DATA/", file_name_extension = "SOHO_MERGE_DATA_LIST.csv",
    unicode = "utf-8", columnTF = False)

### 3. 행정동코드 - 상권코드 매핑 데이터
```
KCD_MERGE_LIST[0] = KCD_MERGE_LIST[0].drop(["상권_코드_2022"], axis = 1)
KCD_MERGE_LIST[0].drop_duplicates(subset = ["STDG_EMD_CD"], inplace = True)
KCD_MERGE_LIST[0].isnull().sum()
  # 상권코드           431
  # 매출액_변동계수       431
KCD_MERGE_LIST[0].isnull().sum()
  # 상권코드               404
len(MARKET_LOC_MAP["상권_코드_2022"].unique())
  # 1622
len(MARKET_LOC_MAP['STDG_EMD_CD'].unique())
  # 465
```

In [8]:
MARKET_LOC_MAP = mountInstance.loadData(
    file_path = "/content/drive/MyDrive/2023BigContest/data/",
    file_name_extension = "market_loc_map.csv",
    unicode = "euc-kr", columnTF = False)

In [9]:
MARKET_LOC_MAP

,상권_코드_2022,상권_코드_2023,행정동_코드,법정동_코드,STDG_EMD_CD
0,2110008,3110008,11110515,11110101,11110101
1,2110008,3110008,11110515,11110102,11110102
2,2110008,3110008,11110515,11110103,11110103
3,2110008,3110008,11110515,11110104,11110104
4,2110008,3110008,11110515,11110105,11110105
...,...,...,...,...,...
3453,2130102,3130102,11230750,11230110,11230110
3454,2130166,3130166,11380590,11380107,11380107
3455,2130113,3130113,11260690,11260106,11260106
3456,2130124,3130124,11290780,11290138,11290138


### 4. 서울열린데이터광장 상권특성 공공데이터

### 5.분기별 상권특성 공공데이터 전처리

### 6. 분기별 상권별 상권특성 공공데이터 병합

### 7. 소호 신용데이터(KCD) 이상치 처리 및 결측값 대체

* KCD_NUM_LIST : 연속형 변수만 추출하여 이상치를 결측값으로 처리한 데이터의 리스트(IQR)
* KCD_IMPUTED_DATA : 유사도가 높은 컬럼별 결측값을 최근접 이웃 5개 데이터의 평균값으로 대체한 데이터의 리스트(KNN, K = 5)
* KCD_PREPROCESS_DATA : 전처리 완료된 데이터에 상권코드, 상권명, 기준일자 등 인덱스 포함한 데이터의 리스트

In [10]:
KCD_DATA = mountInstance.loadData(
    file_path = "/content/drive/MyDrive/2023BigContest/data/DESILO_PLATFORM_DATA/", file_name_extension = "KCD_DATA.csv",
    unicode = "euc-kr", columnTF = False)

In [11]:
KCD_DATA

,기준일자,상권_식별자,상권코드,배달매출액_변동계수,주말배달매출액_변동계수,손익분기점매출액_변동계수,사업장방문고객수_평균,정규고용인원_평균,주말카드매출액_변동계수,매입액_변동계수,...,경영위기사업장_총합,사업장방문신규고객수_평균,월임대료_변동계수,임대보증금_변동계수,매출액_변동계수,카드매출액_변동계수,사업장방문고객수_변동계수,사업장방문신규고객수_변동계수,상권명,외식업종_중분류
0,202201,DMC(디지털미디어시티)_카페,2120098,2.1558,2.5660,0.5387,6.3708,0.8292,1.3211,0.7483,...,0.0000,6.6692,0.5341,0.5648,0.4986,0.4845,0.7065,0.4988,DMC(디지털미디어시티),카페
1,202201,가락시장역_한식,2120234,1.8341,1.9303,0.4745,4.9360,2.3500,1.4125,0.5770,...,0.0000,3.3360,0.5646,0.5735,1.0273,1.1870,1.0395,1.0085,가락시장역,한식
2,202201,가로수길_술집,2120186,2.4496,2.4495,0.7954,3.3886,1.0431,1.6146,0.8874,...,0.0000,2.9971,1.0291,1.1673,0.8737,0.9827,1.1755,0.9964,가로수길,술집
3,202201,가로수길_일식,2120186,2.4495,2.4495,0.3542,6.6600,1.0431,0.3888,0.6522,...,0.0000,5.9857,0.3630,0.4541,0.4905,0.4996,0.8916,0.8940,가로수길,일식
4,202201,가로수길_한식,2120186,2.0289,2.0816,0.2583,9.8900,1.0431,1.0227,0.6220,...,0.0000,6.4200,0.2461,0.3202,0.6021,0.6021,0.7941,0.7777,가로수길,한식
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2687,202303,홍대입구역(홍대)_한식,3120103,2.0752,1.8920,0.7577,7.5168,1.6514,1.0790,1.0970,...,3.0000,5.7888,0.6365,0.7507,0.9257,1.0410,0.7307,0.7528,홍대입구역(홍대),한식
2688,202303,선릉역_카페,3120210,2.1606,2.2517,0.5606,7.7299,1.2906,0.9155,0.7616,...,1.0000,5.3143,0.5803,0.6383,0.7098,0.7216,0.7822,0.7560,선릉역,카페
2689,202303,"신촌역(신촌역, 신촌로터리)_카페",3120094,2.1606,2.2517,0.5606,7.7299,1.2906,0.9155,0.7616,...,0.0000,5.3143,0.5803,0.6383,0.7098,0.7216,0.7822,0.7560,"신촌역(신촌역, 신촌로터리)",카페
2690,202303,"총신대입구역(이수, 총신대)_카페",3120156,2.1606,2.2517,0.5606,7.7299,1.2906,0.9155,0.7616,...,2.0000,5.3143,0.5803,0.6383,0.7098,0.7216,0.7822,0.7560,"총신대입구역(이수, 총신대)",카페


### 8. 분기별 상권데이터와 월별 소호 신용데이터를 오아시스비즈니스 행정동코드에 맞추어 분기별로 변환

In [12]:
MARKET_2022_GROUPED = mountInstance.loadData(
    file_path = "/content/drive/MyDrive/2023BigContest/data/DESILO_PLATFORM_DATA/", file_name_extension = "MARKET_2022_JOIN_GROUPED.csv",
    unicode = "utf-8", columnTF = False)

In [13]:
# MARKET_2022_GROUPED.drop("2022년도_상권_코드", axis = 1, inplace = True)
MARKET_2022_GROUPED[['기준_년_코드', '기준_분기_코드']] = MARKET_2022_GROUPED[['기준_년_코드', '기준_분기_코드']].astype("str")
MARKET_2022_GROUPED["기준일자"] = MARKET_2022_GROUPED['기준_년_코드'] + MARKET_2022_GROUPED['기준_분기_코드'] + "Q"

In [14]:
MARKET_2023_GROUPED = mountInstance.loadData(
    file_path = "/content/drive/MyDrive/2023BigContest/data/DESILO_PLATFORM_DATA/", file_name_extension = "MARKET_2023_JOIN_GROUPED.csv",
    unicode = "utf-8", columnTF = False)

In [15]:
MARKET_2023_GROUPED[['기준_년_코드', '기준_분기_코드']] = MARKET_2023_GROUPED[['기준_년_코드', '기준_분기_코드']].astype("str")
MARKET_2023_GROUPED["기준일자"] = MARKET_2023_GROUPED['기준_년_코드'] + MARKET_2023_GROUPED['기준_분기_코드'] + "Q"

In [16]:
class QuartertoMonth():
  def __init__(self, KCD_dataframe, MARKET_2022, MARKET_2023, KCD_COLUMNS = None):
    self.KCD_dataframe = KCD_dataframe
    self.KCD_COLUMNS = KCD_COLUMNS
    self.MARKET_2022 = MARKET_2022
    self.MARKET_2023 = MARKET_2023

    self.dataframe_month_list = []
    self.dataframe_quarter_list = []
    self.dataframe_market_list = []

  def splitKCDQuarterData(self, base_date: str, date_time1, date_time2,
                       remove_columns: List, groupby_standard_columns: List):
    # 월별 소호 신용데이터를 분기별로 만들기 위한 집계 연산(GROUP BY)을 수행할 컬럼
    self.KCD_COLUMNS = self.KCD_dataframe.columns.tolist()
    for column in remove_columns:
      self.KCD_COLUMNS.remove(column)

    # 월별 시계열을 원소로 담는 리스트 컴프리핸션
    date_time_list = [str(date_time1 + n) for n in range(0, 12, 1)]
    date_time_list.extend([str(date_time2 + n) for n in range(0, 3, 1)])
    date_time_list = [int(date_time) for date_time in date_time_list]

    # 데이터를 월별로 분할 후 분기별로 병합하여 멤버변수로 저장
    for i in range(len(self.KCD_dataframe)):
      self.dataframe_month_list.append(
          self.KCD_dataframe[ self.KCD_dataframe[base_date].isin(date_time_list[i:i+3]) ])
    for index, quarter in zip( list(range(0, 5, 1)), ["20221Q", "20222Q", "20223Q", "20224Q", "20231Q"]):
      index_data = self.dataframe_month_list[index].groupby(groupby_standard_columns)[self.KCD_COLUMNS].sum()
      index_data.reset_index(inplace = True)
      index_data["기준분기"] = quarter
      self.dataframe_quarter_list.append(index_data)
      index_data = 0
    return self.dataframe_quarter_list

  def splitMARKETuarterData(self, base_date: str):
    for index in self.MARKET_2022[base_date].unique():
      self.dataframe_market_list.append(self.MARKET_2022[self.MARKET_2022[base_date] == index])
    self.dataframe_market_list.append(self.MARKET_2023)
    return self.dataframe_market_list

  def seasonalPattern(self, base_date: str):
    SEASONAL_PATTERN = self.KCD_dataframe.groupby([base_date])[self.KCD_COLUMNS].mean()
    SEASONAL_PATTERN = SEASONAL_PATTERN / SEASONAL_PATTERN.mean()
    SEASONAL_PATTERN.reset_index(inplace = True)
    return SEASONAL_PATTERN

In [17]:
mappingTimeseries = QuartertoMonth(KCD_dataframe = KCD_DATA,
                                   MARKET_2022 = MARKET_2022_GROUPED,
                                   MARKET_2023 = MARKET_2023_GROUPED)

In [18]:
KCD_LIST = mappingTimeseries.splitKCDQuarterData(
     base_date = "기준일자",
     date_time1 = 202201, date_time2 = 202301,
     remove_columns = ['기준일자','상권_식별자','상권코드','상권명','외식업종_중분류','소상공인연령_평균','소상공인업력_평균'],
     groupby_standard_columns = ["상권_식별자", "상권코드", "상권명", "외식업종_중분류"])

In [19]:
MARKET_LIST = mappingTimeseries.splitMARKETuarterData(base_date = "기준일자")

In [20]:
SEASON_PATTERN = mappingTimeseries.seasonalPattern(base_date = "기준일자")

In [21]:
pd.set_option("display.max_column", None)

In [247]:
# KCD_LIST[0]["상권코드"], MARKET_LIST[0]["상권_코드"]
# msno.bar(MERGE_LIST[0])

---

In [ ]:
"""
CODE = pd.merge(CONTEST_2022[["STDG_EMD_CD"]], MARKET_LOC_MAP, how = "inner")
# CODE = pd.merge(CONTEST_2023[["STDG_EMD_CD"]], MARKET_LOC_MAP, how = "inner")
KCD_LIST[4]["상권코드"] = KCD_LIST[4]["상권코드"] + 1000000
KCD_LIST[4]
# KCD_MERGE_LIST
KCD_MERGE_LIST = []

for index in range(len(KCD_MERGE_LIST)):
  KCD_MERGE = pd.merge(left = KCD_LIST[index][["상권코드", "매출액_변동계수"]], right = CODE[["상권_코드_2022", "STDG_EMD_CD"]],
                       left_on = "상권코드", right_on = "상권_코드_2022", how = "outer")
  KCD_MERGE_LIST.append(KCD_MERGE)
  KCD_MERGE = 0

for index in range(len(KCD_MERGE_LIST)):
  KCD_MERGE_LIST[index] = KCD_MERGE_LIST[index].drop(["상권_코드_2022", "매출액_변동계수"], axis = 1)
  KCD_MERGE_LIST[index].drop_duplicates(subset = ["STDG_EMD_CD"], inplace = True)
  KCD_MERGE_LIST[index].reset_index(drop = True, inplace = True)
  print(KCD_MERGE_LIST[index].isnull().sum())
  # KCD_MERGE_LIST[0][KCD_MERGE_LIST[0]["상권코드"].isna()]

# display(KCD_MERGE_LIST[4])

KCD_JOIN_LIST = []

for index, quarter in zip( list(range(0, 5, 1)), ["20221Q", "20222Q", "20223Q", "20224Q", "20231Q"]):
  join_data = pd.merge(left = KCD_MERGE_LIST[index], right = KCD_LIST[index], on = "상권코드", how = "left")
  # join_data.drop_duplicates(subset = ["STDG_EMD_CD"], inplace = True)
  join_data.reset_index(drop = True, inplace = True)
  join_data["기준분기"] = quarter
  KCD_JOIN_LIST.append(join_data)
  print(len(KCD_JOIN_LIST[index]))
  # print(KCD_JOIN_LIST[index].isnull().sum())

MERGE_LIST = []

for index in range(0, 5, 1):
  merge_data = pd.merge(
      left = KCD_JOIN_LIST[index], right = MARKET_LIST[index],
      left_on = "상권코드", right_on = "상권_코드",
      how = "left")
  MERGE_LIST.append(merge_data)

MERGE_LIST[0]
KCD_20221Q_20224Q_JOINDATA = pd.concat(MERGE_LIST[0:4], axis = 0)
"""

In [ ]:
"""
CODE = pd.merge(CONTEST_2023[["STDG_EMD_CD"]], MARKET_LOC_MAP, how = "inner")
KCD_LIST[4]["상권코드"] = KCD_LIST[4]["상권코드"] + 1000000
KCD_LIST[4]
KCD_MERGE_LIST = []

for index in range(len(KCD_LIST)):
  KCD_MERGE = pd.merge(left = KCD_LIST[index][["상권코드", "매출액_변동계수"]], right = CODE[["상권_코드_2023", "STDG_EMD_CD"]],
                       left_on = "상권코드", right_on = "상권_코드_2023", how = "outer")
  KCD_MERGE_LIST.append(KCD_MERGE)
  KCD_MERGE = 0

for index in range(len(KCD_MERGE_LIST)):
  KCD_MERGE_LIST[index] = KCD_MERGE_LIST[index].drop(["상권_코드_2023", "매출액_변동계수"], axis = 1)
  KCD_MERGE_LIST[index].drop_duplicates(subset = ["STDG_EMD_CD"], inplace = True)
  KCD_MERGE_LIST[index].reset_index(drop = True, inplace = True)
  print(KCD_MERGE_LIST[index].isnull().sum())
  # KCD_MERGE_LIST[0][KCD_MERGE_LIST[0]["상권코드"].isna()]

display(KCD_MERGE_LIST[4])

KCD_JOIN_LIST = []

for index, quarter in zip( list(range(0, 5, 1)), ["20221Q", "20222Q", "20223Q", "20224Q", "20231Q"]):
  join_data = pd.merge(left = KCD_MERGE_LIST[index], right = KCD_LIST[index], on = "상권코드", how = "left")
  # join_data.drop_duplicates(subset = ["STDG_EMD_CD"], inplace = True)
  join_data.reset_index(drop = True, inplace = True)
  join_data["기준분기"] = quarter
  KCD_JOIN_LIST.append(join_data)
  print(len(KCD_JOIN_LIST[index]))
  # print(KCD_JOIN_LIST[index].isnull().sum())


MERGE_LIST = []

for index in range(0, 5, 1):
  merge_data = pd.merge(
      left = KCD_JOIN_LIST[index], right = MARKET_LIST[index],
      left_on = "상권코드", right_on = "상권_코드",
      how = "left")
  MERGE_LIST.append(merge_data)

MERGE_LIST[4]
KCD_20231Q_JOINDATA = MERGE_LIST[4]
"""

In [203]:
# pd.concat([KCD_20221Q_20224Q_JOINDATA, KCD_20231Q_JOINDATA], axis = 0).to_csv("/content/drive/MyDrive/2023BigContest/KCD_MARKET_CONTEST_JOIN.csv", index = False)
# SEASON_PATTERN.to_csv("/content/drive/MyDrive/2023BigContest/KCD_SEASON_PATTERN.csv", index = False)

### 9. 분기별 상권데이터를 소호 신용데이터를 활용하여 월별로 확장(계절성 지수)

In [22]:
KCD_MARKET_DATA = mountInstance.loadData(
    file_path = "/content/drive/MyDrive/2023BigContest/data/", file_name_extension = "KCD_MARKET_CONTEST_JOIN.csv",
    unicode = "utf-8", columnTF = False)

In [ ]:
"""
KCD_MARKET_DATA.iloc[0:613, -1] = "20221Q"
KCD_MARKET_DATA.iloc[613:1226, -1] = "20222Q"
KCD_MARKET_DATA.iloc[1226:1843, -1] = "20223Q"
KCD_MARKET_DATA.iloc[1843:2463, -1] = "20224Q"
KCD_MARKET_DATA.iloc[2463:, -1] = "20231Q"

KCD_MARKET_DATA.isnull().sum()
  상권코드              1350
  STDG_EMD_CD         21
  상권_식별자            1350
  상권명               1350
  외식업종_중분류          1350
                    ...
  운영_영업_개월_평균       1350
  폐업_영업_개월_평균       1350
  서울_운영_영업_개월_평균    1350
  서울_폐업_영업_개월_평균    1350
  기준일자                 0

KCD_MARKET_DATA.insert(0, "기준일자", KCD_MARKET_DATA.pop("기준일자"))
KCD_MARKET_DATA
KCD_MARKET_DATA.to_csv("/content/drive/MyDrive/2023BigContest/KCD_MARKET_CONTEST_JOIN.csv", index = False)
"""